In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 코사인 유사도 분석

In [2]:
cabDF = pd.read_csv('./project/cabinet.csv', index_col=0)
cabDF['image'] 

0        ./static/img/kitchen/cabinet/cab1.png
1        ./static/img/kitchen/cabinet/cab2.png
2        ./static/img/kitchen/cabinet/cab3.png
3        ./static/img/kitchen/cabinet/cab4.png
4        ./static/img/kitchen/cabinet/cab5.png
                        ...                   
100    ./static/img/kitchen/cabinet/cab104.png
101    ./static/img/kitchen/cabinet/cab105.png
102    ./static/img/kitchen/cabinet/cab106.png
103    ./static/img/kitchen/cabinet/cab107.png
104    ./static/img/kitchen/cabinet/cab108.png
Name: image, Length: 105, dtype: object

In [4]:
# 명사 추출
from konlpy.tag import Okt

okt = Okt()

box = []
for i in tqdm(cabDF['review']):
    a = okt.nouns(i)
    # 하나의 문장으로 합쳐지게 join 시킴
    box.append(' '.join(a))

cabDF['nouns'] = box

100%|██████████| 105/105 [00:14<00:00,  7.10it/s]


In [5]:
cabDF.head(1)

,brand,item,price,image,link,review,nouns
0,아리아퍼니쳐,Tribeca 주방/거실 높은 수납장 1000+상부장추가선택,"159,000",./static/img/kitchen/cabinet/cab1.png,https://ohou.se/productions/452975/selling?aff...,['디자인 너무 깔끔하고 서랍도 부드럽게 잘 열리고 닫히고 수납도 넉넉해서 완전 맘...,디자인 서랍 수납 완전 맘 듭니 군더더기 그 자체 완전 실물 더 실용 최고 이전 수...


In [6]:
# 중간 저장
cabDF.to_csv('cabDF.csv')

In [7]:
cabDF = pd.read_csv('./cabDF.csv', index_col=0)
cabDF.head(1)

,brand,item,price,image,link,review,nouns
0,아리아퍼니쳐,Tribeca 주방/거실 높은 수납장 1000+상부장추가선택,"159,000",./static/img/kitchen/cabinet/cab1.png,https://ohou.se/productions/452975/selling?aff...,['디자인 너무 깔끔하고 서랍도 부드럽게 잘 열리고 닫히고 수납도 넉넉해서 완전 맘...,디자인 서랍 수납 완전 맘 듭니 군더더기 그 자체 완전 실물 더 실용 최고 이전 수...


In [8]:
# 단어 카운트 기반의 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(cabDF['nouns']) # dtm : 텍스트를 단어의 등장 횟수로 표현된 문서-단어 행렬(document-term matrix)
dtm_cab = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names_out()) # dtm.toarray() : 희소행렬인 dtm을 밀집행렬로 변환
dtm_cab

,가걱,가게,가격,가구,가구라,가구점,가까이,가끔,가나,가능,...,흠집,흠칩,흠칫,희망,흰색,힐것,힐링,힘껏,힘들엇어,힘좀
0,0,0,9,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,9,6,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,0,0,9,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0,1,7,5,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0,2,13,10,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,0,0,2,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
101,0,0,4,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
102,0,0,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# dtm 파일 저장
dtm_cab.to_csv('dtm_cab.csv')

---

In [11]:
import pandas as pd 
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma

cabDF = pd.read_csv('./cabDF.csv', index_col=0)
dtm_cab = pd.read_csv('./dtm_cab.csv', index_col=0)

# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
# a, b : 두 벡터
def make_cosine(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

kkma = Kkma()
text = '앉아도 꺼지지 않는 폭식한 소파'
strlist = kkma.nouns(text)
text = [' '.join(strlist)]

result = cv.transform(text) # 위에서 fit 시켰기 때문에 transform만 해주면 됨
input_text = result.toarray()

brand_list = np.array(cabDF['brand'])
item_list = np.array(cabDF['item'])
price_list = np.array(cabDF['price'])
image_list = np.array(cabDF['image'])

result_cosine = make_cosine(dtm_cab, input_text.reshape(-1))
result_args = result_cosine.argsort()[::-1] # argsort() : 배열을 정렬했을 때의 인덱스를 반환하는 함수

# 내림차순 정렬 => [result_args][0] : 코사인 유사도가 가장 큼
print(brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0])

아이리스코리아  스틸 손잡이 트롤리 틈새 수납장 선반 공간박스 책장 홈카페장 카트 2size 27,900 ./static/img/kitchen/cabinet/cab10.png


In [12]:
# 위의 코드 함수화
def find_cab(text):

    import pandas as pd 
    import numpy as np
    from konlpy.tag import Okt
    from konlpy.tag import Kkma

    cabDF = pd.read_csv('./cabDF.csv', index_col=0)
    dtm_cab = pd.read_csv('./dtm_cab.csv', index_col=0)

    # 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
    def make_cosine(a,b):
        return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

    kkma = Kkma()
    search_text = [text]
    strlist = kkma.nouns(search_text[0])
    search_text = [(" ".join(strlist))]

    result = cv.transform(search_text)
    search_words = result.toarray()

    brand_list = np.array(cabDF['brand'])
    item_list = np.array(cabDF['item'])
    price_list = np.array(cabDF['price'])
    image_list = np.array(cabDF['image'])

    result_cosine = make_cosine(dtm_cab, search_words.reshape(-1))

    result_args = result_cosine.argsort()[::-1]
    return brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0]